# PyBADS Example 1: Basic usage

In this introductory example, we will show a simple usage of Bayesian Adaptive Direct Search (BADS) to perform optimization of a synthetic target function.

This notebook is Part 1 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [ ]:
from tkinter import FALSE
import numpy as np
from pybads.bads.bads import BADS

## 0. What is (Py)BADS?

BADS is a fast Bayesian optimization algorithm designed to solve difficult optimization problems, in particular related to parameter estimation -- aka model fitting -- of computational models (e.g., via maximum-likelihood or maximum-a-posteriori estimation). **PyBADS is its Python implementation**.

BADS has been intensively tested for fitting a variety of computational models, and is currently used by many research groups around the world (see [Google Scholar](https://scholar.google.co.uk/scholar?cites=7209174494000095753&as_sdt=2005&sciodt=0,5&hl=en) for many example applications). In our benchmark with real model-fitting problems, BADS performed on par or better than many other common and state-of-the-art optimizers, as shown in the [original BADS paper](https://arxiv.org/abs/1705.04405).

BADS is recommended when no gradient information is available, and the objective function is non-analytical or noisy, for example evaluated through numerical approximation or via simulation.
It requires no specific tuning and runs off-the-shelf like other built-in optimizers (e.g., from `scipy.optimize`).

*Note*: If you are interested in estimating posterior distributions (i.e., uncertainty and error bars) over model parameters, and not just point estimates, you might also want to check out Variational Bayesian Monte Carlo for Python (PyVBMC), a package for Bayesian posterior and model inference which can be used in synergy with PyBADS.

## 1. Problem definition

Here we show PyBADS at work on [Rosenbrock's banana function](https://en.wikipedia.org/wiki/Rosenbrock_function) in 2D as target function.

We specify wide hard bounds and tighter plausible bounds that (hopefully) contain the solution. 
- Hard lower/upper bounds `lb`, `ub` are the actual optimization bounds; PyBADS will not evaluate the target function outside these bounds, but might evaluate them on the bounds.
- Plausible lower/upper bounds `plb`, `pub` represent our best guess at bounding the region where the solution might lie. The plausible bounds do not change the optimization problem, but help define starting points and hyperparameters of PyBADS.

We set a starting point for the optimization $\mathbf{x}_0 = (0, 0)$.

In [2]:
from pybads.function_examples import rosenbrocks_fcn
target = rosenbrocks_fcn;

lb = np.array([[-20, -20]])     # Lower bounds
ub = np.array([[20, 20]])       # Upper bounds
plb = np.array([[-5, -5]])      # Plausible lower bounds
pub = np.array([[5, 5]])        # Plausible upper bounds
x0 = np.array([[0, 0]]);        # Starting point

## 2. Initialize and run the optimization

Then, we initialize a `bads` instance which takes care of the optimization. For now, we use default options.  
To run the optimization, we simply call `bads.optimize()`.

In [3]:
bads = BADS(target, x0, lb, ub, plb, pub)
x_min, fval = bads.optimize()

Variables (index) defined with periodic boundaries: []
Beginning optimization of a DETERMINISTIC objective function

 Iteration f-count     f(x)     MeshScale     Method     Actions
     0         3       1.000000      1.000000            Uncertainty test
     0         7       1.000000      1.000000     Initial mesh       Initial points
     0        11       1.000000      0.500000     Refine grid       Train
     1        19       1.000000      0.250000     Refine grid       Train
     2        21       0.532334      0.250000     Successful search (('ES-ell', 1))       
     2        22       0.205350      0.250000     Successful search (('ES-ell', 1))       
     2        23       0.031554      0.250000     Successful search (('ES-ell', 1))       
     2        25       0.000405      0.250000     Incremental search (('ES-ell', 1))       
bads:_robust_gp_fit_: posterior GP update failed. Singular matrix for L Cholesky decomposition
bads:_robust_gp_fit_: posterior GP update failed. Si

In [50]:
print(f"BADS minimum at: x = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {bads.function_logger.func_count}, time: {round(bads.optim_state['total_time'], 2)} s")

BADS minimum at: x = [1.00111511 1.00266865], fval = 2.036e-05
total f-count: 64, time: 4.28 s


The true minimum is at $\textbf{x}^\star = [1, 1]$, where $f_\text{min} = 0$.  

In conclusion, PyBADS found the solution with a very small number of function evaluations (`f-count`), which is particularly important if the target function is mildly-to-very expensive to compute as in many computational models.

*Note*: PyBADS by default does not aim for extreme numerical precision of the target (e.g., beyond the 2nd or 3rd decimal place), since in most realistic model-fitting problems a higher resolution is typically pointless, e.g. due to noise or variability in the data.